# Handling MultiCollinearity

#### 1.Correlation
#### 2.variance inflation factor

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
cars = pd.read_csv('datasets/cars_processed_new.csv')
cars.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Origin,Age
0,18.000000,8,307.0,130,3504,12.0,US,49
1,16.000000,8,304.0,150,3433,12.0,US,49
2,17.000000,8,302.0,140,3449,10.5,US,49
3,14.000000,8,454.0,220,4354,9.0,US,49
4,23.551429,8,440.0,215,4312,8.5,US,49


In [4]:
cars.describe()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Age
count,387.000000,387.000000,387.000000,387.000000,387.000000,387.000000,387.000000
mean,23.672514,5.410853,192.184755,103.645995,2965.387597,15.573643,42.917313
std,7.736579,1.667795,103.703706,38.128651,846.332848,2.746260,3.668715
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,37.000000
25%,17.600000,4.000000,102.500000,75.000000,2221.500000,13.900000,40.000000
50%,23.200000,4.000000,146.000000,92.000000,2790.000000,15.500000,43.000000
75%,29.000000,6.000000,260.000000,121.000000,3589.500000,17.050000,46.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,49.000000


In [10]:
from sklearn import preprocessing

cars['MPG'] = preprocessing.scale(cars[['MPG']].astype('float64'))
cars['Cylinders'] = preprocessing.scale(cars[['Cylinders']].astype('float64'))
cars['Displacement'] = preprocessing.scale(cars[['Displacement']].astype('float64'))
cars['Horsepower'] = preprocessing.scale(cars[['Horsepower']].astype('float64'))
cars['Weight'] = preprocessing.scale(cars[['Weight']].astype('float64'))
cars['Acceleration'] = preprocessing.scale(cars[['Acceleration']].astype('float64'))
cars['Age'] = preprocessing.scale(cars[['Age']].astype('float64'))

In [11]:
cars.describe()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Age
count,387.000000,3.870000e+02,3.870000e+02,3.870000e+02,3.870000e+02,3.870000e+02,3.870000e+02
mean,0.000000,3.672055e-17,3.672055e-17,1.101617e-16,1.101617e-16,2.203233e-16,8.078522e-16
std,1.001294,1.001294e+00,1.001294e+00,1.001294e+00,1.001294e+00,1.001294e+00,1.001294e+00
min,-1.898967,-1.447404e+00,-1.199046e+00,-1.513838e+00,-1.600007e+00,-2.761372e+00,-1.615000e+00
25%,-0.785926,-8.470340e-01,-8.659368e-01,-7.522710e-01,-8.800918e-01,-6.102152e-01,-7.962160e-01
50%,-0.061154,-8.470340e-01,-4.459295e-01,-3.058349e-01,-2.075007e-01,-2.685060e-02,2.256768e-02
75%,0.689501,3.537065e-01,6.547792e-01,4.557326e-01,7.383860e-01,5.382839e-01,8.413513e-01
max,2.967354,1.554447e+00,2.537570e+00,3.318176e+00,2.572779e+00,3.363956e+00,1.660135e+00


In [12]:
from sklearn.model_selection import train_test_split

x = cars.drop(['MPG', 'Origin'], axis=1)
y = cars['MPG']

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size =0.2)

In [15]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression(normalize=True).fit(x_train, y_train)

In [16]:
print('Training Score: ', linear_model.score(x_train, y_train))

Training Score:  0.7871741885644601


In [18]:
y_pred = linear_model.predict(x_test)

In [20]:
from sklearn.metrics import r2_score

print('Testing score: ', r2_score(y_test, y_pred))

Testing score:  0.7835653828513398


In [21]:
def adjusted_r2(r_square, labels, features):
    adj_r2 = 1 - ((1-r_square) * (len(labels)-1)) / (len(labels) - features.shape[1] - 1)
    return adj_r2

In [31]:
print(' R2 Score : ', adjusted_r2(r2_score(y_test, y_pred), y_test, x_test))

 R2 Score :  0.7652751335148333


In [32]:
features_corr = x.corr()
features_corr

,Cylinders,Displacement,Horsepower,Weight,Acceleration,Age
Cylinders,1.000000,0.922633,0.811466,0.873029,-0.458161,0.321850
Displacement,0.922633,1.000000,0.894199,0.932822,-0.526901,0.357047
Horsepower,0.811466,0.894199,1.000000,0.863388,-0.670920,0.404458
Weight,0.873029,0.932822,0.863388,1.000000,-0.397181,0.299049
Acceleration,-0.458161,-0.526901,-0.670920,-0.397181,1.000000,-0.292705
Age,0.321850,0.357047,0.404458,0.299049,-0.292705,1.000000


In [35]:
abs(features_corr) > 0.8

,Cylinders,Displacement,Horsepower,Weight,Acceleration,Age
Cylinders,True,True,True,True,False,False
Displacement,True,True,True,True,False,False
Horsepower,True,True,True,True,False,False
Weight,True,True,True,True,False,False
Acceleration,False,False,False,False,True,False
Age,False,False,False,False,False,True


In [36]:
trimmed_features = x.drop(['Cylinders', 'Displacement', 'Weight'], axis=1)

In [38]:
trimmed_features_corr = trimmed_features.corr()
trimmed_features_corr

,Horsepower,Acceleration,Age
Horsepower,1.000000,-0.670920,0.404458
Acceleration,-0.670920,1.000000,-0.292705
Age,0.404458,-0.292705,1.000000


In [39]:
abs(trimmed_features_corr)> 0.8

,Horsepower,Acceleration,Age
Horsepower,True,False,False
Acceleration,False,True,False
Age,False,False,True


In [41]:
!pip install statsmodels

     |████████████████████████████████| 8.1MB 4.3MB/s eta 0:00:01
     |████████████████████████████████| 235kB 12.6MB/s eta 0:00:01


In [43]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [45]:
vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
vif['Features'] = x.columns
vif

# 1       Not Correlated with other features
# 1- 5    Moderately correlated with other features
# >5      Highly correlated with other features

,VIF Factor,Features
0,6.841102,Cylinders
1,16.099578,Displacement
2,8.820275,Horsepower
3,10.691363,Weight
4,2.493882,Acceleration
5,1.223454,Age


In [46]:
x = x.drop(['Displacement', 'Weight'], axis =1)
vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
vif['Features'] = x.columns
vif

,VIF Factor,Features
0,3.048996,Cylinders
1,4.559951,Horsepower
2,1.895344,Acceleration
3,1.196806,Age


In [47]:
x = cars.drop(['MPG', 'Origin', 'Displacement', 'Weight'], axis=1)
y = cars['MPG']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [49]:
linear_model = LinearRegression(normalize=True).fit(x_train, y_train)
print('Training Score:', linear_model.score(x_train, y_train))

Training Score: 0.7180583160240379


In [50]:
y_pred = linear_model.predict(x_test)
print('R2 Score: ', r2_score(y_test, y_pred))

R2 Score:  0.7506296360402803


In [51]:
print(' R2 Score : ', adjusted_r2(r2_score(y_test, y_pred), y_test, x_test))

 R2 Score :  0.7369655065082409
